# Big Data Processes Exercises - Week 07

## <font color = pink>Data cleaning  & Exploratory Data Analysis</font>

#### What we will cover today:

1. Importing packages and libraries
2. Loading and examining the dataset
3. Exploratory Data Analysis
4. Removing unnecessary columns
5. Data cleaning
6. Pearson
    1. Heatmap
7. Distribution and Visualisations
8. Outliers
9. Features with biggest impact

***
***
***

## 1. Importing various libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import asarray

***
***
## 2. Loading the dataset

We will be working with the IBM-Employee-Attrition dataset yet again. However this time, **<font color=Orchid>we will be working with the <i>'raw'</i>, original dataset </font>** - Straight Outta Kaggle.

In the raw dataset, 'Attrition' is actually 'Yes' and 'No' and not '1' and '0'. During this Notebook you will learn how to change such instances into numerical data.

In [ ]:
df = pd.read_csv("IBM.csv", delimiter = ",")

In [ ]:
df

### **<font color= turquoise>EXERCISE 1</font>**
Examine the dataset throughly. What are some other differences between this 'raw' dataset and the cleaned IBM-dataset, we have worked with so far? For easier comparison, load the other IBM-Employee-Attrition dataset into the Notebook.

Write your answer here:

***
***
## 3. Exploratory Data Analysis

We will be working with the 'Attrition' column - our target variable. We want to figure out whether an employee has Attrition or not, i.e., whether the employee left the company or not.

Since we will be working with the 'Attrition' feature, let's start by moving it to the front column in the dataset for better visability.

*If you want more information about the columns and the values behind the columns, we refer to the Kaggle page from which this dataset is from https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset/data and https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset/discussion/139552*

In [ ]:
# We copy the column, drop (i.e., delete) it from our dataframe and insert it to the front
front = df['Attrition']
df.drop(labels = ['Attrition'], axis=1, inplace=True)
df.insert(0, 'Attrition', front)

#displaying quick overview for the dataset
df.head()


Below are examples of bits of code that you can use for initial exploration/getting an overview of the dataset.

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# Checking all unique values
df.nunique()

***
***

## 4. Removing unnecessary columns (features)

Above, we called the nunique() method. As you can see from the output, some columns have few unique values. For instance, 'Attrition' has only two different values ('Yes'  and 'No'). 'EducationField' on the other hand, has five unique values ('Life Sciences', 'Other', 'Medical', 'Marketing', 'Technical Degree', 'Human Resources'). Finally, 'EmployeeNNumber' has 1470 unique values. This is because each employee has their unique employeee number and the dataset has information on 1470 different employees.

As you can see from the examples above, the number of unique values vary from column to column. When a column (feature) has many unique and distinct values, it is deemed to have **high cardinality.** 

- High cardinality is when a feature has many values. A boolean column, which only can have the values of <font color=pink> true or false </font>, has a cardinality of 2. HTTP status codes – <font color=pink> 200, 301, 302, 404, 500 </font> – might have a cardinality under a few dozen. But, for an online shopping system, fields like <font color=pink> userId, shoppingCartId, and orderId </font> are often high-cardinality columns that can take take hundreds of thousands of distinct values. 
A large number of distinct values, tend to provide more detail for observation. On the other hand, attributes with low cardinality, having only a few distinct values, may limit .the level of insight that can be obtained. However, it is safe to say, that *very* high cardinality does not provide us with better insight or give our model a pattern for it's prediction.

Selecting a column and using .unique() method gives us unique values within that variable:

In [ ]:
df['Attrition'].unique() # cardinality of 2

In [ ]:
df['EducationField'].unique() # cardinality of 6

In [ ]:
df['EmployeeNumber'].unique() # cardinality of 1470 as every employee has their own number (N.B.: some numbers, like 3, are missing from the dataset)

### 4.1 Features with high/low cardinality

Let us take a closer look at some of the the columns with a suspiciously high or low number of unique values. The reason we want to look at these is, that it is important to somehow deal with the features with high cardinality - either by transforming the values or completely deleting the column. 

OBS! Remember to write that in your reports as to why you're deleting and/or transforming your data, as a simple action as that can introduce bias in your model! 

In [ ]:
df['EmployeeCount'].unique() # this column has only one value: '1'

In [ ]:
df['MonthlyRate'].unique() # this column has a lot of different values

In [ ]:
df['Over18'].unique() # this column too has only one value: 'Y'

In [ ]:
df['StandardHours'].unique() # this column too has only one value: '80'

**EmployeeCount, Over18, and StandardHours** contain only one unique value. 


- **EmployeeCount:** We cannot find more information about the column EmployeeCount on Kaggle - we can only assume it describes the number of employees represented in a given observation, which won't add anything to our prediction, since it is the same value ('1') across all observations.
- **Over18:** Every value in this column is 'Y', meaning the employee is over 18 years of age. This won't add to our prediction of Attrition either.
- **StandardHours:** Every value in this column is '80'. The unit here is unclear and no more information on Kaggle is provided.

Since all of the abovementioned values are the same, we can confidently remove the columns, as they will be useless in helping us predicting our target.

**EmployeeNumber** is a somewhat different story:

- **EmployeeNumber:** As described, this column holds the personal number of the given employee. It is probably some kind of ID. This column won't help us predict attrition, since it only contains "a number describing a person". In other words, there is no pattern that will emerge from this column that will help us predict attrition. 

Remember! This is our own assumption! Remember to note and write down everything you do with the dataset and why in your project reports!

Also! <font color=pink> *If you want to double-check which columns can be dropped, create a pearson correlation matrix! Columns with 'NaN' are a good indicator that they can be dropped. Hint hint: Notebook 3 BDP_Classification.ipynb. However! This does not apply to columns like 'EmployeeNumber' since it contains multiple unique values and not a single value*

In [ ]:
# deleting the columns 'EmployeeNumber', 'EmployeeCount', 'Over18' and 'StandardHours' and saving the result in our variable 'df'
df = df.drop(['EmployeeNumber', 'EmployeeCount', 'Over18', 'StandardHours'],  axis=1)

In [ ]:
# checking that the columns has indeed been deleted

df.columns

***
***
## 5. Data Cleaning


### 5.1 Handling nominal/categorical data

A lot of the features/columns in the IBM dataset contain nominal/categorical data (if you are confused by the terms 'nominal', 'ordinal' etc., take a look in this week's text 'Exploratory Data Analysis' by Victoria Cox). 

To use nominal data for modeling, we need to transform it to numerical data, more on how to do this in a minute (or take a look here: https://towardsdatascience.com/handling-categorical-data-the-right-way-9d1279956fc6). 

First, let's look at the different types of nominal data.

Nominal data can be:
- *non-ordinal*, i.e. categories that are *not* ordered. In other words, the catgories have no natural, internal ordering. An example is a variable such as 'EducationField' which holds the following categories: 'Life Sciences', 'Other', 'Medical', 'Marketing', 'Technical Degree', 'Human Resources'
- *ordinal*, i.e. categories that have a natural order. An example *could* be a variable such as 'Education'. Here bachelor is below master which is below phd. However, be aware that this is *our* assumption. In your projects, you should explain the reasons for deciding that a nominal variable contains ordinality. You should also explain your choice of encoding, i.e., why you have chosen to handle the nominal variables in your dataset in the way you have.

When we handle nominal/categorical data, we need to assess whether the data is non-ordinal or ordinal, because these two types of data should often be handeled differently. 

**Binary, nominal data** which hold only two categories (i.e., the binary variables) can be handled using something called dummy coding. Dummy variables are variables that are either 0 or 1. Dummy coding entails assigning one variable the value '0' and the other one value '1'. Often, 0 signifies false/no/none while 1 signifies true/yes. For example, if we wanted to dummy code the nominal variable 'car type' (containing the two categories 'conventional car' and 'electric car'), we could create a column called 'Electric'. We would then set the variable to '0' for cars running on gas and '1' for cars running on electricity.

In the IBM dataset we have two variables, which can be dummy coded in this way:
- 'Attrition': 'Yes', 'No'
- 'Gender': 'Male', 'Female'

**Non-ordinal, nominal data** data which hold three or more categories can be handled with *one-hot encoding*. Here, one new binary variable is added for each category in the variable. In other words, one-hot encoding creates one binary variable for each category.

Examples of non-ordinal, nominal data in the IBM dataset is:
- 'EducationField', 'JobRole', 'MaritalStatus'

**One-hot encoding, visualized:**
<br>
![title](one-hot.png)

**Ordinal nominal data** can, in some cases, be replaced by numbers, which signify the natural ordering. For this, the .replace() method can be used.

In the IBM dataset, much of the ordinal categories have already been converted into numbers (see the dataset documentation on Kaggle to figure out what the original categories were: https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset/data):
- 'Education'
- 'JobInvolvement'
- 'JobLevel'
- 'JobSatisfaction'
- 'RelationshipSatisfaction'
- 'WorkLifeBalance'
- 'PerformanceRating'

However, be aware that one need to be careful when replacing categories with numbers. When replacing categories with numbers, we assume a meaningful, sequential ordering of the categories. Moreover, we also assume that the categories are 'evenly spaced'. And, any models we train on this data, will assume this as well.

If we look at a variable such as 'Education', which consists of 'bachelor', 'master' and 'phd', these categories are not 'evenly spaced'. A bachelor is usually 3 years (4 in the US), a master is 2 years and a phd 3 years (4 in the us). Thus, the encoding made in the current data (bachelor = 1, master = 2, phd = 3) makes some 'wrong' assumptions, such as: master ('2') is the double of a bachelor ('1') and phd ('3') is three times a bachelor. This might be a problem when training your model. 

Another, more meaninful, way of encoding the 'Education' variable, could be to code the years of education:
- bachelor = '4', signifying that the degree takes 4 years of education (as the data is from the US)
- master = '6', signifying that the degree takes 6 years of education (4 from the bachelor + 2 from the master)
- phd = '10', signifying that the degree takes 10 years of education (4 from the bachelor + 2 from the master + 4 from the phd)

Thus, we can critisize the people who encoded the IBM dataset variable 'Education' and argue that they should either code it based om years spent on education OR one-hot encode it instead.

We highlight this, because you will need to make this type of decisions when handling the nominal/categorical data in your own dataset. We encourage you to experiment with your nominal encoding and see how it affects the models you train.

Below, we use .replace() to handle 'Attrition', 'Gender' and 'BusinessTravel'.

Next, we handle 'EducationField', 'JobRole', 'MaritalStatus' using one-hot encoding (usig the method .get_dummies()).



In [ ]:
df.replace({"Attrition" : {"No": 0, "Yes" : 1}}, inplace=True)
df.replace({"Gender" : {"Female": 0, "Male" : 1}}, inplace=True)
df.replace({"OverTime" : {"No": 0, "Yes" : 1}}, inplace=True)
df.replace({"BusinessTravel" : {"Non-Travel": 0, "Travel_Rarely": 1, "Travel_Frequently" : 2}}, inplace=True)

In [ ]:
df = pd.get_dummies(df, columns = ["EducationField", "Department", "JobRole", "MaritalStatus"], dtype='int') 

df

In [ ]:
# Check which columns our dataset currently holds (after we handled the nominal variables)
df.columns
#df.shape

Now, we want to take a look at the dataset, post encoding:

In [ ]:
# To see all columns, we first ask Pandas to display all columns:
pd.set_option('display.max_columns', None)   

# Next, we take a look at the transformed dataset
df


As you can se, we have now succesfully transformed the columns with nominal data.
The columns now all contain numbers. We can confirm this by calling .dtypes. As the output shows, all columns now contain integer values:

In [ ]:
df.dtypes

### 5.2 Handling missing values

Besides from handling nominal values, data cleaning also consists of finding and handling missing values. For more on how to do this, take a lok at the notebook from week 2.


***
***
## 6. Pearson

A little recap from week 3 - BDP_Classification.ipynb.

In order to build any model we must specify a **target variable/dependent variable** and one or more **feature variables/independent variables**.

- **(X)** = feature or independent variable
- **(y)** = target or dependent variable - in this case, 'Attrition'

In order to figure out what variables to choose as feature(s), aka, which variables we will use to try to predict our target, we create a **correlation matrix**. 

The correlation matrix we will create visualizes the **pearson correlation** between the variables in the dataset. Other correlation methods include: 
- Spearman 
- Kendall 
- Gamma 
- Eta 
- **You can look up the different correlation types and how they differ on the internet**

In [ ]:
# `corr`: The computed correlation matrix
# Method = pearson
# Numeric values only. A correlation cannot take into account qualitative data unless they are transformed, which we just did above!
corr = df.corr(method='pearson', numeric_only=True)
corr

### 6.1 Heatmap

We have constructed a pearson correlation matrix, however for better readability we will create a **heatmap** based on our correlation matrix. A heatmap is basically a graphical representation of the data that uses color coding to represent different values. 


Here we use the library Seaborn which we defined as 'sns'.

**<font color=Orchid>REMEMBER: If you'd like to inspect the heatmap using zoom, hover your mouse over the plot and click on the middle button (the one looking like a small histogram) in the upper right corner.</font>**

In [ ]:
# We create a new figure (fig) and axis (ax) with a specified size.
fig, ax = plt.subplots(figsize=(50,50)) # Adjust the figsize values to change the size of each cell in the heatmap.

# - `xticklabels` & `yticklabels`: Use the columns of the correlation matrix for labeling the x and y axis respectively.
#   blue represents positive correlations and red represents negative correlations.
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap='RdBu_r', annot=True, linewidth=0.5, ax=ax)

The Pearson correlation coefficient measures the linear relationship between two variables.
Its value ranges from -1 to 1, where:
- **1** implies a perfect **positive linear relationship**. If one variable increases, the other variable increases as well
- **-1** implies a perfect **negative linear relationship**. When one variable increases, the other variables decreases
- **0** implies **no relationship** between the variables
- The darker the colour, the more correlated the variables are

<font color=Orchid>OBS! Remember that correlation is not causation!</font>

### 6.2 Sorting the heatmap

Since we only want to focus on attrition, lets us sort our axes with their correlation with attrition.

In [ ]:
#Sorting the axes by thier correlation with attrition

col = df.corr().nlargest(48, "Attrition").Attrition.index
plt.figure(figsize=(50, 50))
sns.heatmap(df[col].corr(), vmin=-0.3, vmax=0.3, cmap='RdBu_r', annot=True, linewidth=0.5, annot_kws={"size":10}, fmt='.1g')


### **<font color= turquoise>EXERCISE 2</font>**

What can we conclude from this correlation matrix?
And how may the information in the matrix help us in our quest to predict attrition?

**Hints** 
- 'Attrition': 0 = no, the employee did not leave, 1 = yes the employee left
- Red signifies a positive correlation = attrition is positively correlated with the given feature
- Blue signifies a negative correlation = attrition is negatively correlated with the given feature
- The darker the color, the more strongly correlated the variables are 

***
***
## 7. Distribution & Visualisation

We will now look at the distribution within the different features and visualise them based on Victoria Cox's chapter. 

We will do through "basic visualisations" - but we urge you to do some research and find other types of plots and visualisations. Pairplots, barplots, boxplots, etc...

### 7.1 Age column

Below, we count the number of employees of different ages. For instance, we can see that there is 8 employees in the dataset, who are 18 years old.

In [ ]:
# Cheap way to check consistency and distribution within a column
df.groupby(['Age']).Age.count()

Now, we visualize the age distribution:

In [ ]:
df['Age'].value_counts().sort_index().plot(kind='bar', rot=0, ylabel='count', figsize=(15,5))

plt.show()

# You can also write a for-loop to view all the distributions of the columns without writing out all the colum names
#for col in df:
#    df[col].value_counts().sort_index().plot(kind='bar', rot=0, ylabel='count')
#    plt.show()
#Only a requirement outside of Jupyter Notebook

Pay attention to any odd distribution - they can say a lot about the data quality, and *hint hint, missing values!*

### **<font color=turquoise>EXERCISE 3</font>**

Is there anything odd about the 'Age' distribution? What about other features? Any odd distributions there?

Write your answers here

### 7.2 Attrition Column

Now you know how to visualise and analyse the distribution of a features. It's also important to see the distribution of our target feature - Attrition.

*Little recap from Evaluation week - BDP_Evaluation.ipynb*

In [ ]:
#To better see the number of 'Yes' and 'No' values
df.groupby(['Attrition']).Attrition.count()

Now let's visualise the distribution of Attrition!

In [ ]:
df['Attrition'].value_counts().sort_index().plot(kind='bar', rot=0, ylabel='count')
plt.show()


We can see that 237 employees have somehow left the company. 

237 ÷ 1233 = 0.1922 = 19.22%

This means that we are working with an imbalanced dataset, and during our modelling phase, we have to have that in mind and take the imbalance into account i.e. when choosing a correct metric.

***
***

## 8. Outliers

Outliers (i.e., extreme values like '155' in an 'Age'-column) can be wrong values in a dataset. Either due to a typo, conversions, or just due to the context of the dataset.

One way to look for outliers, is to look at the min and max value of a given feature/column. Do they make sense in the context of the database and in our real world? Remember! These are assumptions! 



In [ ]:
df.describe()

### **<font color= turquoise>EXERCISE 4</font>**
Can you find the age outliers? Outliers can mess with the result of our model. A way to deal with them is to transform them - either by inserting another value or by removing them from the dataset entirely. 

*Hint hint!* Go back to the Notebook from week 02 - BDP_Basics, to see how to remove a spefic row.

Write your answer here:

### **<font color= turquoise>EXERCISE 5</font>**
What are the consequences of transforming an outlier or completely removing it? How does it affect modelling? Are there any ethical concerns doing this?

Write your answer here:

***
***

## 9. Which variables have the biggest impact?

We know from our Pearson Correlation Matrix and Pearson Correlation Heatmap that some features are more strongly correlated to our target feature than others. Obs! Again, remember that correlation is not causation! We need to figure out which features actually have the most impact on 'Attrition', because it is these features that can help us better predict Attrition.

We will now create a histogram over the Pearson Correlation results.

In [ ]:
# We create a histogram, but we don't want Attrition there, since that is our target and it would just be correlated with it self.
df.drop('Attrition', axis=1).corrwith(df.Attrition).sort_values().plot(kind='barh', figsize=(10, 10))

All these variables have a correlation with attrition -- either positive or negative. The variables in the middle, not as much. Therefore, we can start by training a model on  cut the features that have the strongest correlation to attrition

### **<font color= turquoise>EXERCISE 6</font>**
Examine whether more or less features is better when training classification models using machine learning. 

***
***
***

# Take home messages

After finishing this notebook, you should know how to:
- thoroughly examine a dataset
- encode values
- modify a Pearson Correlation heatmap
- visualise and analyse a distribution
- Find and handle outliers
- Decide on which features to use when predicting your target variable

## Time to explore and clean your own datasets!

## Good luck with your projects!
## - Your BDP TAs  ❤